# Импорт библиотек и модулей для расчета 

In [101]:
import numpy as np 
import sys
import os
from scipy import optimize
import CoolProp.CoolProp as CP
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
from pathlib import Path
import glob
from scipy.interpolate import interp1d
from sklearn.metrics import mean_squared_error

# Импорт класса расчета 
from class_DpDz import DpDz

In [102]:
folder_path = '/Users/andrey/Desktop/NIR/Datasets/Graph9/*.csv'
all_files = glob.glob(folder_path)

dataframes = []
for f in all_files:
    # Читаем файл
    df = pd.read_csv(f, sep='\t')
    
    # Переименовываем колонки и конвертируем в Па/м
    df = df.rename(columns={'X': 'x', 'Y': 'Target'})
    df['Target'] = df['Target'] * 1000
    
    # Извлекаем значение G из имени файла: 'Gr3.csv' → 3 → 300
    filename = os.path.basename(f)
    valueG = int(filename[2]) * 100
        
    df['G'] = valueG
    
    dataframes.append(df)

combined_dfG = pd.concat(dataframes, ignore_index=True)

In [103]:
G = np.array(sorted(combined_dfG['G'].unique()))
x = [combined_dfG[combined_dfG['G'] == g]['x'].values for g in G]
research_1 = []

for i, g in enumerate(G):
        mode = {
                'Substance': 'CO2',
                'Temperature': -10,
                'x': x[i],
                'G': g
                }
        research_1.append(mode)

In [104]:
first_experimental_setup = []
for research_param in research_1:
    fes = DpDz(g=0, ki=None, d=0.00142, value_fb=False, thermodynamic_params=research_param)
    first_experimental_setup.append(fes)

In [105]:
results_of_first_research = []
for i, fes in enumerate(first_experimental_setup):
    res = fes.calculate()
    dat = pd.DataFrame(res)
    results_of_first_research.append(dat)

resultG = pd.concat(results_of_first_research, ignore_index=True)

In [106]:
resultG = resultG.drop(columns = ['B', 'Substance', 'Re liquid', 'Re gas', 'fi', 'alpha', 'Pred','Liquid density', 'Gas density',	'Lquid viscosity', 'Gas viscosity',	'Simplex density', 'Simplex viscosity'])
resultG = pd.concat([resultG, combined_dfG['Target']], axis=1)

In [107]:
folder_path = '/Users/andrey/Desktop/NIR/Datasets/Graph12/*.csv'
all_files = glob.glob(folder_path)

dataframes = []
for f in all_files:
    # Читаем файл
    df = pd.read_csv(f, sep='\t')
    
    # Переименовываем колонки и конвертируем в Па/м
    df = df.rename(columns={'X': 'x', 'Y': 'Target'})
    df['Target'] = df['Target'] * 1000
    
    # Извлекаем значение T из имени файла: '10.csv' → 10
    filename = os.path.basename(f)
    valueT = int(filename.replace('.csv', ''))
        
    df['T'] =  - valueT
    
    dataframes.append(df)

combined_dfT = pd.concat(dataframes, ignore_index=True)
combined_dfT = combined_dfT.sort_values(by=['T', 'x'], ascending=[False, True]).reset_index(drop=True)

In [108]:
T = np.array(sorted(combined_dfT['T'].unique()))
x = [combined_dfT[combined_dfT['T'] == t]['x'].values for t in T]
research_2 = []

for i, t in enumerate(T):
        mode = {
                'Substance': 'CO2',
                'Temperature': t,
                'x': x[i],
                'G': 300
                }
        research_2.append(mode)

In [109]:
second_experimental_setup = []
for research_param in research_2:
    ses = DpDz(g=0, ki=None, d=0.00142, value_fb=False, thermodynamic_params=research_param)
    second_experimental_setup.append(ses)

In [110]:
results_of_second_research = []
for i, fes in enumerate(second_experimental_setup):
    res = fes.calculate()
    dat = pd.DataFrame(res)
    results_of_second_research.append(dat)

resultT = pd.concat(results_of_second_research, ignore_index=True)

In [111]:
resultT = resultT.drop(columns = ['B', 'Substance', 'Re liquid', 'Re gas', 'fi', 'alpha', 'Pred','Liquid density', 'Gas density', 'Lquid viscosity', 'Gas viscosity'])
resultT = resultT.sort_values(by=['T', 'x'], ascending=[False, True]).reset_index(drop=True)
resultT = pd.concat([resultT, combined_dfT['Target']], axis=1)

In [112]:
resultT.to_csv('/Users/andrey/Desktop/NIR/resultT.csv', index=False)
resultG.to_csv('/Users/andrey/Desktop/NIR/resultG.csv', index=False)